In [ ]:
import rioxarray
min_date = '2022-09-01' # Start time
max_date = '2023-10-01' # End time
thuanhoa_path = "ThuanHoa/region/ST_ThuanHoa_Boundaryofficially.shp"
VH_file = "ThuanHoa/ThuanHoa_VH.tif"
VV_file = "ThuanHoa/ThuanHoa_VV.tif"

In [ ]:
%%time
# Basic plots
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
from datacube.utils import masking
from datacube.utils.cog import write_cog
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map, rgb
from dea_tools.datahandling import mostcommon_crs

# EASI defaults
easinotebooksrepo = '/home/kaito/CSIROBoeingPhase4-Vietnam/easi-notebooks'
if easinotebooksrepo not in sys.path: sys.path.append(easinotebooksrepo)
from easi_tools import EasiDefaults, xarray_object_size, notebook_utils, unset_cachingproxy
from easi_tools.load_s2l2a import load_s2l2a_with_offset

# Data tools
import numpy as np
from datetime import datetime

# Datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-algo/blob/main/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-algo/blob/main/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
# import cartopy.crs as ccrs
# from datashader import reductions
from holoviews import opts
from utils import load_data_geo
import rasterio
import os
import rioxarray
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

from deafrica_tools.bandindices import calculate_indices
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import joblib

LOAD NDVI

In [47]:
import json
import xarray as xr

output_dir = 'data_file'
output_file = os.path.join(output_dir, 'data.nc')

# Function to convert JSON strings back to dictionaries
def restore_json_attrs(data):
    for var_name, variable in data.variables.items():
        for attr_name, attr_value in variable.attrs.items():
            if isinstance(attr_value, str):
                try:
                    # Attempt to convert the JSON string back to a dictionary
                    variable.attrs[attr_name] = json.loads(attr_value)
                except json.JSONDecodeError:
                    # If it fails, it's not a JSON string, so keep it as is
                    pass

# Load the data
data = xr.open_dataset(output_file)

# Restore the attributes
restore_json_attrs(data)

# Now you can use the dataset with the restored attributes
print(data['blue'][0]['y'])
# display(data)




<xarray.DataArray 'y' (y: 532)> Size: 4kB
array([1072225., 1072215., 1072205., ..., 1066935., 1066925., 1066915.])
Coordinates:
    time         datetime64[ns] 8B 2022-09-02T03:35:38.706000
  * y            (y) float64 4kB 1.072e+06 1.072e+06 ... 1.067e+06 1.067e+06
    spatial_ref  int32 4B ...
Attributes:
    units:       metre
    resolution:  -10.0
    crs:         EPSG:32648


In [28]:
flag_name = 'scl'
flag_desc = masking.describe_variable_flags(data[flag_name])  # Pandas dataframe
display(flag_desc)
display(flag_desc.loc['qa'].values[1])

,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


{'0': 'no data',
 '1': 'saturated or defective',
 '2': 'dark area pixels',
 '3': 'cloud shadows',
 '4': 'vegetation',
 '5': 'bare soils',
 '6': 'water',
 '7': 'unclassified',
 '8': 'cloud medium probability',
 '9': 'cloud high probability',
 '10': 'thin cirrus',
 '11': 'snow or ice'}

In [29]:
# Create a "data quality" Mask layer
flags_def = flag_desc.loc['qa'].values[1]
good_pixel_flags = [flags_def[str(i)] for i in [2, 4, 5, 6]]  # To pass strings to enum_to_bool()

# enum_to_bool calculates the pixel-wise "or" of each set of pixels given by good_pixel_flags
# 1 = good data
# 0 = "bad" data

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> DataArray
# display(good_pixel_mask)  # Type: bool

In [39]:
print(data[flag_name])

<xarray.DataArray 'scl' (time: 78, y: 532, x: 830)> Size: 34MB
array([[[255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255],
        ...,
        [255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255]],

       [[255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255],
        ...,
        [255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255]],

       ...,

       [[255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255],
        ...,
        [255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255]],

       [[255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255],
        ...,
        [255, 255, ..., 255, 255],
        [255, 255, ..., 255, 255]]], dtype=uint8)
Coordinates:
  * time         (time) datetime64[ns] 624B 2022-09-02T03:35:38.706000 ... 20...
  * y            (y) float64 4kB 1.072e+06 1.072e+06 ... 1.067e+06 1.067e+06
  * x            (x) float64 7kB 5.95e+05 5.95e+05 ... 6.033e+05 6.033e+05
    spatial_ref  int32 4B ...
Attributes:
   

In [ ]:
data_layer_names = [x for x in data.data_vars if x != 'scl']
rs = []
for layer_name in data_layer_names:
    # Apply valid mask (calculated above) and good pixel mask
    # layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask)
    layer = data[[layer_name]].where(good_pixel_mask)
    rs.append(layer)
    
# Calculate intermediate result
result = xr.merge(rs).persist()

display(result)

In [ ]:
ds1 = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = ds1["NDVI"]
average_ndvi = ndvi.resample(time='1M').mean().persist()  ## tính mean cho từng tháng -> time = 13
average_ndvi  # DataArray

In [ ]:
average_ndvi = average_ndvi.compute()
plt.imshow(average_ndvi.isel(time=5))